# Proyecto 1. Calendario laLiga

#### Partidos

In [1]:
import pandas as pd
from itertools import permutations

df = pd.read_csv('../Data/equipos_data.csv', index_col = 0)
equipos = list(df['equipo'].unique())
partidos = list(permutations(df['equipo'].unique(), 2))

#### Días

In [2]:
import pickle
filename = '../Data/list_calendar.pkl'
lista_dias = open(filename, 'rb')
with open(filename, 'rb') as f:
    lista_dias=pickle.load(f)
# lista_dias

In [3]:
import pickle
filename = '../Data/dic_dias_jugables_r.pkl'
dic_dias_reales = open(filename, 'rb')
with open(filename, 'rb') as f:
    dic_dias_reales=pickle.load(f)

#### Finde

In [4]:
import pickle
filename = '../Data/list_day.pkl'
es_finde = open(filename, 'rb')
with open(filename, 'rb') as f:
    es_finde=pickle.load(f)
# Es finde? 1 si, 0 no

### Modelo

In [5]:
filename = '../Data/predicciones.pickle'
predicciones = open(filename, 'rb')
with open(filename, 'rb') as f:
    predicciones=pickle.load(f)

## Modelado

### Variables

https://developers.google.com/optimization/assignment/assignment_example

In [6]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('CP-SAT') #CBC, SCIP, SAT

x = {}
for i in partidos:
    for j in lista_dias:
        x[i, j] = solver.BoolVar(f'Partido {i} jugado el dia {j}')

### Variables ficticias

In [7]:
dl = {}
for i in partidos: 
    dl[i] = solver.IntVar(0, len(lista_dias)-1, f'Día lectivo en el que se juega {i}')
    solver.Add(dl[i] == solver.Sum([dia * x[i,j] for dia, j in enumerate(lista_dias)]))

In [8]:
dr = {}
for i in partidos: 
    dr[i] = solver.IntVar(0, 295, f'Día real en el que se juega {i}')
    solver.Add(dr[i] == solver.Sum([dia * x[i,j] for dia, j in dic_dias_reales.items()]))

### Restricciones

<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [9]:
for j in lista_dias:
    solver.Add(solver.Sum([x[i, j] for i in partidos])<=6)

<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [10]:
for i in partidos:
    solver.Add(solver.Sum([x[i, j] for j in lista_dias])==1)

<div class="alert alert-info">
  <strong>Las cadenas de TV prefieren que los partidos sean el finde</strong>
</div>

In [11]:
solver.Add(solver.Sum([x[i, lista_dias[j]] for j in range(len(lista_dias)) if es_finde[j] for i in partidos])>=0.501*380)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001C7BC8186F0> >

<div class="alert alert-info">
  <strong>Separar los partidos de ida y vuelta (mínimo 60 días jugables)</strong>
</div>

In [12]:
for i in partidos:
    for indice, j in enumerate(lista_dias[:len(lista_dias)-50+1]):
        solver.Add(solver.Sum([x[i, j1]+x[(i[1],i[0]), j1] for j1 in lista_dias[indice:indice+50+1]])<=1)

<div class="alert alert-info">
  <strong>Es deseable que no se jueguen demasiados partidos seguidos en casa o como visitante</strong>
</div>

In [13]:
for equipo1 in equipos:
    for dia in range(1, len(lista_dias)+1):
        restriccion = solver.Constraint(-2, 2, '')
        for j in lista_dias[:dia]:
            for equipo2 in equipos:
                if equipo1 != equipo2:
                    restriccion.SetCoefficient(x[(equipo1, equipo2), j], 1)
                    restriccion.SetCoefficient(x[(equipo2, equipo1), j], -1)

<div class="alert alert-info">
  <strong>Máximo de 7 días sin jugar un mismo equipo</strong>
</div>

In [14]:
for equipo in equipos:
    for j,fecha in enumerate(lista_dias):
        if j < 139:
            restriccion = solver.Constraint(1,solver.Infinity(), f'Máximo numero de días sin jugar {equipo}, fecha {j}')
            for aux in range(1,8):
                k=j+aux-1

                for i in partidos:
                    if equipo in i:
                        restriccion.SetCoefficient(x[i, lista_dias[k]], 1)

<div class="alert alert-info">
  <strong>No muchos partidos seguidos del mismo equipo (mínimo 3 días entre partido y partido)</strong>
</div>

In [15]:
for e in equipos:
    for i in partidos:
        for i_aux in partidos:
            if e in i and e in i_aux and i!=i_aux and not (i[0] == i_aux[1] and i[1] == i_aux[0]):
                Y_pos = solver.BoolVar(f'Partido {i} vs partido {i_aux} Y positiva')
                Y_neg = solver.BoolVar(f'Partido {i} vs partido {i_aux} Y negativa')
        
                solver.Add(solver.Sum([Y_pos,Y_neg]) == 1)
                solver.Add(-1000000*Y_pos + dr[i]-dr[i_aux]>= -1000000 +3)
                solver.Add(1000000*Y_neg + dr[i]-dr[i_aux]<= 1000000 -3)

### Función objetivo

In [16]:
objective_terms = []
for i in partidos:
    for j in lista_dias:
        objective_terms.append(predicciones[i,j] * x[i, j])
solver.Maximize(solver.Sum(objective_terms))

### Resultado

In [17]:
from time import time

t0 = time()

solver.SetTimeLimit(540 * 60 * 1000) # Maximo de tiempo en ms
solver.SetNumThreads(6)
status = solver.Solve()

t1 = time() - t0

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Audiencia total = {int(solver.Objective().Value())} espectadores\n')
    print(f'Audiencia media = {int(solver.Objective().Value()/380)} espectadores\n')
    cont = 0
    for i in partidos:
        for j in lista_dias:
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                cont+=1
                print(f'Partido {i} jugado el día {j}.' +
                      f' Audiencia esperada: {int(predicciones[(i,j)])} espectadores')
                
    print(f'\nPartidos totales jugados: {cont}')
    
else:
    print('Sin solución.')

Sin solución.


In [18]:
print(f'Ha tardado {t1 // 3600} horas, {t1 % 3600 // 60} minutos y {round(t1 % 60, 2)} segundos')

Ha tardado 9.0 horas, 1.0 minutos y 45.9 segundos


In [19]:
status

6

### Comprobación de restricciones

"results" es una matriz en el que las filas son los partidos y las columnas los día. results[i,j] = 1 si se juega el partido i el día j. Como es una matriz no se puede indexar con texto, por lo que "i" y "j" son el índice que corresponde a ese día y a es partido. Por ejemplo, el i = 0 corresponde al partido ('Real Madrid CF', 'FC Barcelona'). Si tienes el nombre del partido o el día y lo quieres pasar a índice puedes utilizar "partido_a_ind" o "dia_a_ind".

In [20]:
import pandas as pd
import numpy as np

dia_a_ind = {dia : i for i, dia in enumerate(lista_dias)}
partido_a_ind = {p : i for i, p in enumerate(partidos)}
results = np.zeros((len(partidos), len(lista_dias)))

for k,v in x.items():
    i, j = k
    i = partido_a_ind[i]
    j = dia_a_ind[j]
    
    results[i, j] = v.solution_value()
    
# int(round(v.solution_value()))

In [21]:
import pickle

with open('../Data/results.pkl', 'wb') as file:
    pickle.dump(results, file)

<div class="alert alert-info">
  <strong>Día lectivo</strong>
</div>

In [22]:
bien = True
for partido, dia_lectivo in dl.items():
    i = partido_a_ind[partido]
    j = round(dia_lectivo.solution_value())

    if results[i, j] < 0.5:
        print('La restricción no va bien')
        bien = False
        break
    
if bien:
    print('La restricción funciona correctamente')

La restricción no va bien


<div class="alert alert-info">
  <strong>Máximo 4 partidos por día</strong>
</div>

In [23]:
suma = np.sum(results, axis = 0) 
if np.all(suma <= 4):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

La restricción funciona correctamente


<div class="alert alert-info">
  <strong>Todos los partidos tienen que jugarse una vez</strong>
</div>

In [24]:
suma = np.sum(results, axis = 1)
if np.all(suma > 0):
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

La restricción no va bien


<div class="alert alert-info">
  <strong>Las cadenas de TV prefieren que los partidos sean el finde</strong>
</div>

In [25]:
prop = np.sum(results)*0.6
finde = np.array(es_finde) == True
partidos_por_dias = np.sum(results,axis=0)

if np.sum(partidos_por_dias[finde]) >= prop:
    print('La restricción funciona correctamente')
else: print('La restricción no va bien')

La restricción funciona correctamente


<div class="alert alert-info">
  <strong>Separar los partidos de ida y vuelta (mínimo 60 días jugables)</strong>
</div>

In [26]:
for partido in partidos:
    ida = results[partido_a_ind[partido]]
    vuelta = results[partido_a_ind[partido[1], partido[0]]]
    dia_ida = np.where(ida == 1)[0][0]
    dia_vuelta = np.where(vuelta == 1)[0][0]
    if abs(dia_ida - dia_vuelta) < 60:
        print('La restricción no va bien')
else:
    print('La restricción funciona correctamente')

IndexError: index 0 is out of bounds for axis 0 with size 0

<div class="alert alert-info">
  <strong>Es deseable que no se jueguen demasiados partidos seguidos en casa o como visitante</strong>
</div>

In [ ]:
try:
    for equipo1 in equipos:
        partidos_equipo1 = np.zeros(len(lista_dias))
        for equipo2 in equipos:
            if equipo1 != equipo2:
                local = partido_a_ind[equipo1, equipo2]
                vis = partido_a_ind[equipo2, equipo1]
                partidos_equipo1 += results[local] - results[vis]
        
        aux = 0
        for p in partidos_equipo1:
            aux += p
            if aux < -1 or aux > 1:
                raise RuntimeError("No va")
                
except RuntimeError:
    print('La restricción no va bien')
    
else:
    print('La restricción funciona correctamente')

<div class="alert alert-info">
  <strong>No muchos partidos seguidos del mismo equipo (mínimo 3 días entre partido y partido)</strong>
</div>

In [ ]:
from datetime import datetime
for e in equipos:
    for i in partidos:
        # COMPRUEBA IDAS
        if i[0] == e:
            # Iteramos cada par de partidos de ese equipo
            for i_aux in partidos:
                if i[1] != i_aux[1] and i_aux[0] == e: # Omite el mismo partido
                    indice_dia = np.where(results[partido_a_ind[i]] == 1)[0][0]
                    indice_dia_aux = np.where(results[partido_a_ind[i_aux]] == 1)[0][0]

                    #Saca fechas de ambos partidos
                    for fecha, v in dia_a_ind.items(): 
                        if v == indice_dia:
                            f1 = fecha
                        if v == indice_dia_aux:
                            f2 = fecha
                    # Comprobamos que pasan al menos tres dias
                    f_date = datetime.strptime(f1,'%d-%m-%Y')
                    l_date = datetime.strptime(f2,'%d-%m-%Y')
                    delta = abs(f_date - l_date)
                    if delta.days <3:
                        print('La restricción no va bien')
        
        # COMPRUEBA VUELTAS
        if i[1] == e:

            for i_aux in partidos:
                if i[0] != i_aux[0] and i_aux[1] == e: # Omite el mismo partido
                    indice_dia = np.where(results[partido_a_ind[i]] == 1)[0][0]
                    indice_dia_aux = np.where(results[partido_a_ind[i_aux]] == 1)[0][0]
                    #Saca fechas de ambos partidos
                    for fecha, v in dia_a_ind.items():  
                        if v == indice_dia:
                            f1 = fecha
                        if v == indice_dia_aux:
                            f2 = fecha
                    # Comprobamos que pasan al menos tres dias
                    f_date = datetime.strptime(f1,'%d-%m-%Y')
                    l_date = datetime.strptime(f2,'%d-%m-%Y')
                    delta = abs(f_date - l_date)
                    if delta.days <3:
                        print('La restricción no va bien')
else:
    print('La restricción funciona correctamente')

<div class="alert alert-info">
  <strong>Máximo de 5 días sin jugar un mismo equipo</strong>
</div>

In [ ]:
for j,_ in enumerate(lista_dias):
    if j < 140:
        for e in equipos:
            lista_partidos_equipo = []
            suma = 0
            for i in partidos:
                if e in i:
                    lista_partidos_equipo.append(partido_a_ind[i])
            for arr in results[lista_partidos_equipo]: # partidos de ese equipo
                suma += np.sum(arr[j:j+5]) #añadimos los partidos de ese equipo que se juegan en esos días
            if suma < 1:
                print('La restriccion no funciona correctamente')
else: print('La restriccion funciona correctamente')  